In [ ]:
%load_ext autoreload
%autoreload 2
from get_mmap import get_mmap
import random

# Setup the necessary paths and parameters
fmap = 'e5_all_bat_fb_2mhz_n500_beta.raw'
data_title = 'Stochastic A6 BAT FB'
path_data = 'results/'
name_data = 'a6_bat_fb_stochastic_f17_all_dlinear_n1000r_p1_slr_g1000_r10.pkl'
rand_iter = 10
total_values = 2**16
pmul = 2**4
n_profile = 1000*pmul # ensure that: n_profile + n_attack*nr_attack_groups < nr_trials
n_attack = 150
n_profile_per_group = 100
nr_traces_vec = [1, 2, 5, 10, 20, 50, 100]

random.seed(0)

In [ ]:
# Load file
print('Mapping data...')
mmap_data, metadata = get_mmap(fmap)
print('Done mapping data')

In [ ]:
import numpy as np

print('Extracting values corresponding to mapped data')
D_all_bytes = mmap_data['data'][0]['B'][1:3, :].T
D_all = np.bitwise_xor(np.left_shift(D_all_bytes[:, 0].astype(np.uint16), 8), D_all_bytes[:, 1].astype(np.uint16))

In [ ]:
print('Selecting traces for profile and attack...')
nr_trials = metadata['nr_trials']
total_trials_per_value = nr_trials // total_values
idx_group = np.arange(1, total_trials_per_value + 1)

idx_profile_group = np.where(idx_group % 4 == 0)[0]
total_profile_trials_per_group = len(idx_profile_group)
idx_attack_group = np.union1d(np.union1d(np.where(idx_group % 4 == 1)[0], np.where(idx_group % 4 == 2)[0]), 
                              np.where(idx_group % 4 == 3)[0])

total_profile = total_values * total_profile_trials_per_group
idx_profile_all = np.zeros(total_profile, dtype=int)

si = np.argsort(D_all, axis=0)
for k in range(total_values):
    idx_val = si[total_trials_per_value * k: total_trials_per_value * (k + 1)]
    idx_profile_all[total_profile_trials_per_group * k: total_profile_trials_per_group * (k + 1)] = idx_val[idx_profile_group]

idx_profile_sel = np.random.choice(idx_profile_all, n_profile, replace=False)

In [ ]:
# Select base for coefficients
base = 'F17'

# Select values for discriminant evaluation
V_discriminant = np.arange(total_values)

# Select number of values for V_profile and V_attack
ng_profile = [2**16]
ng_attack = [2**8]
nr_eval = 3

# Set up attack/result cells
nr_attacks = 5
results = np.full((len(ng_attack), len(ng_profile), nr_eval, nr_attacks), None)

In [ ]:
# Run attacks
for r in range(nr_eval):
    for k in range(len(ng_attack)):
        # Select groups for V_attack
        if ng_attack[k] == total_values:
            V_attack = V_discriminant  # full guessing entropy
        else:
            V_attack = np.random.randint(0, total_values, ng_attack[k])

        for i in range(len(ng_profile)):
            print(f'Running attacks for (k,i,r)=({k},{i},{r})')

            # Select groups for V_profile
            if ng_profile[i] == total_values:
                V_profile = np.arange(total_values)
            else:
                V_profile = np.random.randint(0, total_values, ng_profile[i])

            # Run attack for selection, 20ppc
            atype = 'selection'
            aparams = {
                'signal': 'bnorm',
                'sel': '20ppc',
                'p1': 40,
                'p2': 0.90
            }
            discriminant = 'linearfast'
            eparams = {'save_signals': 1}
            results[k, i, r, 0] = run_stochastic_attack_generic_mmap(
                mmap_data, D_all, idx_profile_sel,
                mmap_data, D_all, idx_attack_group,
                V_profile, V_attack, V_discriminant,
                base, atype, aparams, discriminant,
                rand_iter, nr_traces_vec, eparams
            )

            # Run attack for S-PCA (aka Master PCA)
            atype = 'templatepca'
            aparams = {
                'pca_threshold': 0.95,
                'pca_alternate': 0,
                'pca_dimensions': 10
            }
            discriminant = 'linearfast'
            eparams = {}
            results[k, i, r, 1] = run_stochastic_attack_generic_mmap(
                mmap_data, D_all, idx_profile_sel,
                mmap_data, D_all, idx_attack_group,
                V_profile, V_attack, V_discriminant,
                base, atype, aparams, discriminant,
                rand_iter, nr_traces_vec, eparams
            )

            # Run attack for T-PCA (aka Slave PCA)
            atype = 'pca'
            aparams = {
                'idx_traces': idx_profile_group,
                'pca_threshold': 0.95,
                'pca_alternate': 0,
                'pca_dimensions': 10,
                'cov_from_sel': 0
            }
            discriminant = 'linearfast'
            eparams = {}
            results[k, i, r, 2] = run_stochastic_attack_generic_mmap(
                mmap_data, D_all, idx_profile_sel,
                mmap_data, D_all, idx_attack_group,
                V_profile, V_attack, V_discriminant,
                base, atype, aparams, discriminant,
                rand_iter, nr_traces_vec, eparams
            )

            # Run attack for S-LDA (aka Master LDA)
            atype = 'templatelda'
            aparams = {
                'lda_threshold': 0.95,
                'lda_dimensions': 10
            }
            discriminant = 'linearnocov'
            eparams = {}
            results[k, i, r, 3] = run_stochastic_attack_generic_mmap(
                mmap_data, D_all, idx_profile_sel,
                mmap_data, D_all, idx_attack_group,
                V_profile, V_attack, V_discriminant,
                base, atype, aparams, discriminant,
                rand_iter, nr_traces_vec, eparams
            )

            # Run attack for T-LDA (aka Slave LDA)
            atype = 'lda'
            aparams = {
                'idx_traces': idx_profile_group,
                'lda_threshold': 0.95,
                'lda_dimensions': 10,
                'cov_from_sel': 0
            }
            discriminant = 'linearfastnocov'
            eparams = {}
            results[k, i, r, 4] = run_stochastic_attack_generic_mmap(
                mmap_data, D_all, idx_profile_sel,
                mmap_data, D_all, idx_attack_group,
                V_profile, V_attack, V_discriminant,
                base, atype, aparams, discriminant,
                rand_iter, nr_traces_vec, eparams
            )


In [ ]:
import os
import pickle

# Save data to disk so you can prevent wasting time
print('All done, saving data...\n')
os.makedirs(path_data, exist_ok=True)

with open(os.path.join(path_data, name_data), 'wb') as f:
    pickle.dump(results, f)